# S4. Fine-tuning de modelo pre-entrenado para la clasificación de CIFAR-100
En esta sesión veremos como realizar el fine-tuning (ajuste fino) de una arquitectura de red [Inception](https://paperswithcode.com/paper/rethinking-the-inception-architecture-for), más concretamente veremos su [versión 3 (V3) disponible en Keras](https://keras.io/api/applications/inceptionv3/) pre-entrenada con la base de datos de imágenes [ImageNet](https://www.image-net.org) para la clasificación de CIFAR-100. 

### Modelo preentrenado: Inception

La red Inception de Google (2015) basado en CNNs fue uno de los primeros modelos pre-entrenados pensados para que fuera aplicado a otras tareas (*transfer learning*) tras un ajuste fino del modelo. Estos modelos pre-entrenados presentan la arquitectura típica **stem-body-head:**
* **Stem (raíz):** $\;$ dos o tres capas convolucionales que extraen características de bajo nivel
* **Body (cuerpo):** $\;$ subred de **bloques convolucionales** repetidos
* **Head (cabeza):** $\;$ transforma la salida del cuerpo mediante un red densa según la tarea a abordar (clasificación, segmentación, etc.)

<div align="center">
<table><tr>
<td style="border: none;"><img src="Figure_14.19.png"/></td>
</tr></table>
</div>

Esta red toma su nombre de los bloque Inception que se definen en su cuerpo. En cada bloque Inception se concatenan ramas paralelas con kernels diferentes, para que cada capa del cuerpo escoja la que más le convenga
<div align="center">
<table><tr>
<td style="border: none;"><img src="Figure_14.18.png" width=600/></td>
</tr></table>
</div>

### Carga de datos
La utilización de una red pre-entrenada conlleva preprocesar nuestros datos con el mismo preproceso que se empleó para los datos de entrenamiento de la red pre-entrenada. En el caso de la red Inception V3, las imágenes necesitan, entre otras cosas, ser redimensionadas a 299 x 299. Si realizamos este preproceso para nuestro conjunto de entrenamiento, esto requeriría aproximadamente 40GB, por lo que es conveniente realizar este preproceso bajo demanda. Es decir, aplicaremos el preproceso para cada batch en el momento que vaya a ser utilizado.      

La aplicación de este preproceso bajo demanda está ya implementado en el módulo [tensorflow_datasets](https://www.tensorflow.org/datasets), así que en esta sesión cargaremos y manipularemos CIFAR-100 como un objeto [Dataset](https://www.tensorflow.org/api_docs/python/tf/data/Dataset) utilizando este módulo.

In [11]:
import tensorflow as tf
import tensorflow_datasets as tfds
train_data, test_data = tfds.load('cifar100', split=['train', 'test'], as_supervised=True)
train_size = len(train_data)

### Preproceso

Primero definimos una función que aplica el preproceso necesario a una muestra de entrenamiento (imagen, etiqueta de clase). Esto incluye redimensionar la imagen a 299 x 299, el preproceso específico de la red InceptionV3 y convertir la etiqueta de clase a one-hot encoding.

In [12]:
from keras.applications.inception_v3 import preprocess_input

img_size = (299, 299)
num_classes = 100

def preprocess(image, label):
    image = tf.image.resize(image, img_size)
    image = tf.cast(image, tf.float32)
    image = preprocess_input(image)
    label = tf.one_hot(label, num_classes)
    return image, label

A continuación indicamos que la función anteriormente definida se aplicará a cada imagen cuando sea necesario. Para ello utilizamos la función [map()](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#map) del objeto [Dataset](https://www.tensorflow.org/api_docs/python/tf/data/Dataset) y le pasamos como parámetro la función de preproceso que queremos que sea aplique a cada muestra del conjunto de datos.

In [13]:
train_data = train_data.map(preprocess)
test_data = test_data.map(preprocess)

Las funciones [take()](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#take) y [skip()](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#skip) combinadas permiten definir los conjuntos de entrenamiento y validación como nuevos Datasets. 

In [14]:
train_size = int(0.8 * train_size)
train_dataset = train_data.take(train_size)
val_dataset = train_data.skip(train_size)
test_dataset = test_data

print(len(train_dataset),len(val_dataset))

40000 10000


### Carga del modelo pre-entrenado

Seguidamente procedemos con la carga del modelo Inception V3 con los pesos resultantes de entrenarlo con la base de datos Imagenet, pero no queremos que el modelo incluya la capa de salida (include_top=False) que por defecto es una softmax de 1000 clases.

In [15]:
from keras.applications.inception_v3 import InceptionV3

model = InceptionV3(input_shape=img_size + (3,),include_top=False, weights='imagenet')

### Preparación del modelo pre-entrenado

Vamos a preparar la red Inception V3 para ser entrenada (fine-tuning) con CIFAR-100. Dado el número de parámetros de este modelo (21M), nos limitaremos a utilizarlo con los valores por defecto y añadiremos una capa GlobalAveragePooling + MLP seguida de una softmax de 100 neuronas (100 clases) acorde a CIFAR-100 que sí que entrenaremos.

In [16]:
from keras.layers import GlobalAveragePooling2D, Dense, Dropout
from keras.models import Model

for layer in model.layers:
    layer.trainable = False

x = GlobalAveragePooling2D()(model.output)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=model.input, outputs=output)

Compilamos el modelo con los mismos parámetros que en sesiones anteriores.

In [17]:
from keras.optimizers import Adam

opt=Adam(learning_rate=0.001)
model.compile(loss='categorical_crossentropy',
            optimizer=opt,
            metrics=['accuracy'])

Entrenamos el modelo utilizando los conjuntos de datos organizado en batches y que son cargados en memoria dinámicamente.

In [18]:
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from keras.models import load_model

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=0.00001)
checkpoint = ModelCheckpoint(filepath='best_model.h5', monitor='val_accuracy', save_best_only=True, verbose=1)

epochs=10
batch_size=32
train_dataset_batched = train_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
val_dataset_batched = val_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
history = model.fit(train_dataset_batched,
                    epochs=epochs,
                    verbose=1,
                    validation_data=val_dataset_batched,
                    callbacks=[reduce_lr,checkpoint])

Epoch 1/10


2023-12-19 15:25:10.400179: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8906
2023-12-19 15:25:10.510074: W external/local_xla/xla/stream_executor/gpu/asm_compiler.cc:225] Falling back to the CUDA driver for PTX compilation; ptxas does not support CC 8.9
2023-12-19 15:25:10.510087: W external/local_xla/xla/stream_executor/gpu/asm_compiler.cc:228] Used ptxas at ptxas
2023-12-19 15:25:10.510126: W external/local_xla/xla/stream_executor/gpu/redzone_allocator.cc:322] UNIMPLEMENTED: ptxas ptxas too old. Falling back to the driver to compile.
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.
2023-12-19 15:25:16.234261: W tensorflow/compiler/mlir/tools/kernel_gen/transforms/gpu_kernel_to_blob_pass.cc:191] Failed to compile generated PTX with ptxas. Falling back to compilation by driver.
2023-12-19 15:25:16.379626: W tensorflow/compiler/mlir/tools/kernel_gen/transforms/gpu_kerne

   1/1250 [..............................] - ETA: 2:56:50 - loss: 4.9140 - accuracy: 0.0000e+00

2023-12-19 15:25:16.656396: I external/local_xla/xla/service/service.cc:168] XLA service 0x7fbf4cc0d430 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-12-19 15:25:16.656413: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 4060 Laptop GPU, Compute Capability 8.9
2023-12-19 15:25:16.662957: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1702995916.710413    6953 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1250/1250 [==============================] - ETA: 0s - loss: 2.2722 - accuracy: 0.4182

2023-12-19 15:26:53.865320: W tensorflow/core/kernels/data/cache_dataset_ops.cc:858] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.



Epoch 1: val_accuracy improved from -inf to 0.56360, saving model to best_model.h5


/home/jorcisai/.local/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1250/1250 [==============================] - 136s 102ms/step - loss: 2.2722 - accuracy: 0.4182 - val_loss: 1.5402 - val_accuracy: 0.5636 - lr: 0.0010
Epoch 2/10
1250/1250 [==============================] - ETA: 0s - loss: 1.6989 - accuracy: 0.5308
Epoch 2: val_accuracy improved from 0.56360 to 0.59640, saving model to best_model.h5
1250/1250 [==============================] - 123s 98ms/step - loss: 1.6989 - accuracy: 0.5308 - val_loss: 1.4305 - val_accuracy: 0.5964 - lr: 0.0010
Epoch 3/10
1250/1250 [==============================] - ETA: 0s - loss: 1.5728 - accuracy: 0.5612
Epoch 3: val_accuracy improved from 0.59640 to 0.61240, saving model to best_model.h5
1250/1250 [==============================] - 123s 98ms/step - loss: 1.5728 - accuracy: 0.5612 - val_loss: 1.3723 - val_accuracy: 0.6124 - lr: 0.0010
Epoch 4/10
1250/1250 [==============================] - ETA: 0s - loss: 1.4953 - accuracy: 0.5813
Epoch 4: val_accuracy improved from 0.61240 to 0.61890, saving model to best_model.h5


### Cargar el mejor modelo y evaluarlo con el test set

In [19]:
model = load_model('best_model.h5')
test_dataset_batched = test_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
score = model.evaluate(test_dataset_batched, verbose=0)
print(f'Test loss: {score[0]*100:.2f}')
print(f'Test accuracy: {score[1]*100:.2f}')

Test loss: 122.80
Test accuracy: 66.07


### Aumento de datos

En las sesiones anteriores, hemos utilizado la función [ImageDataGenerator](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator) para realizar el aumento de datos. Sin embargo, esta función no se recomienda para los nuevos desarrollos de código por estar obsoleta, y en su lugar se deben utilizar las [capas de preproceso](https://www.tensorflow.org/guide/keras/preprocessing_layers). Más concretamente, utilizaremos algunas de las [capas de preproceso de aumento de datos para imágenes](https://www.tensorflow.org/guide/keras/preprocessing_layers#image_data_augmentation).

Añadiremos estas capas de aumento de datos antes del modelo Inception V3.

In [23]:
from keras.applications.inception_v3 import InceptionV3
from keras.layers import Input, GlobalAveragePooling2D, Dense, Dropout, RandomRotation, RandomTranslation, RandomZoom
from keras.models import Model

input_layer = Input(shape=img_size + (3,))

x = RandomRotation(factor=0.1, fill_mode='nearest')(input_layer)
x = RandomTranslation(height_factor=0.1, width_factor=0.1, fill_mode='nearest')(x)
x = RandomZoom(height_factor=0.2, fill_mode='nearest')(x)

inception_model = InceptionV3(input_shape=img_size + (3,),include_top=False, weights='imagenet')

for layer in inception_model.layers:
    layer.trainable = False

x = inception_model(x)

x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(num_classes, activation='softmax')(x)

aug_model = Model(inputs=input_layer, outputs=output)

In [24]:
from keras.optimizers import Adam

opt=Adam(learning_rate=0.001)
aug_model.compile(loss='categorical_crossentropy',
            optimizer=opt,
            metrics=['accuracy'])

In [25]:
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from keras.models import load_model

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=0.00001)
checkpoint = ModelCheckpoint(filepath='best_model.h5', monitor='val_accuracy', save_best_only=True, verbose=1)

epochs=10
batch_size=32
train_dataset_batched = train_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
val_dataset_batched = val_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
history = aug_model.fit(train_dataset_batched,
                    epochs=epochs,
                    verbose=1,
                    validation_data=val_dataset_batched,
                    callbacks=[reduce_lr,checkpoint])

Epoch 1/10
1250/1250 [==============================] - ETA: 0s - loss: 2.9098 - accuracy: 0.2760
Epoch 1: val_accuracy improved from -inf to 0.49830, saving model to best_model.h5
1250/1250 [==============================] - 132s 104ms/step - loss: 2.9098 - accuracy: 0.2760 - val_loss: 1.8095 - val_accuracy: 0.4983 - lr: 0.0010
Epoch 2/10
1250/1250 [==============================] - ETA: 0s - loss: 2.3303 - accuracy: 0.3827
Epoch 2: val_accuracy improved from 0.49830 to 0.54270, saving model to best_model.h5
1250/1250 [==============================] - 128s 102ms/step - loss: 2.3303 - accuracy: 0.3827 - val_loss: 1.6522 - val_accuracy: 0.5427 - lr: 0.0010
Epoch 3/10
1250/1250 [==============================] - ETA: 0s - loss: 2.2392 - accuracy: 0.4059
Epoch 3: val_accuracy improved from 0.54270 to 0.55660, saving model to best_model.h5
1250/1250 [==============================] - 128s 103ms/step - loss: 2.2392 - accuracy: 0.4059 - val_loss: 1.5704 - val_accuracy: 0.5566 - lr: 0.0010
E

In [26]:
aug_model = load_model('best_model.h5')
test_dataset_batched = test_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
score = aug_model.evaluate(test_dataset_batched, verbose=0)
print(f'Test loss: {score[0]*100:.2f}')
print(f'Test accuracy: {score[1]*100:.2f}')

Test loss: 145.80
Test accuracy: 58.50
